<a href="https://colab.research.google.com/github/manish2021iitd/Deep-Learning/blob/main/DLassignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, random_split
import wandb
from torchvision.datasets import ImageFolder


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [ ]:
wandb.login(key='e3c892d4f8c9cd9b9043d31938ad090f0a32cec1')

In [7]:

#CNN model
class CNN(nn.Module):
    def __init__(self, num_classes, num_filters=32, filter_size=3, dropout_prob=0.0, activation='relu', use_batchnorm=True):
        super(CNN, self).__init__()
        #activation functions
        activation_fn = nn.ReLU() if activation == 'relu' else nn.GELU() if activation == 'gelu' else \
                        nn.SiLU() if activation == 'silu' else nn.Mish()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, num_filters, kernel_size=filter_size, padding='same'),
            nn.BatchNorm2d(num_filters) if use_batchnorm else nn.Identity(),    #batch normalization
            activation_fn,
            nn.MaxPool2d(2,2),
            nn.Conv2d(num_filters, num_filters*2, kernel_size=filter_size, padding='same'),
            nn.BatchNorm2d(num_filters*2) if use_batchnorm else nn.Identity(),
            activation_fn,
            nn.MaxPool2d(2,2),
            nn.Conv2d(num_filters*2, num_filters*4, kernel_size=filter_size, padding='same'),
            nn.BatchNorm2d(num_filters*4) if use_batchnorm else nn.Identity(),
            activation_fn,
            nn.MaxPool2d(2,2),
            nn.Conv2d(num_filters*4, num_filters*8, kernel_size=filter_size, padding='same'),
            nn.BatchNorm2d(num_filters*8) if use_batchnorm else nn.Identity(),
            activation_fn,
            nn.MaxPool2d(2,2),
            nn.Conv2d(num_filters*8, num_filters*16, kernel_size=filter_size, padding='same'),
            nn.BatchNorm2d(num_filters*16) if use_batchnorm else nn.Identity(),
            activation_fn,
            nn.MaxPool2d(2,2)
        )
         #fully connected layers
        self.fc_layers = nn.Sequential(
            nn.Linear(num_filters*16 * 7 * 7, 512),
            activation_fn,
            nn.Dropout(dropout_prob),  #dropout for regularization
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1) #flatten the output for fully connected layers
        x = self.fc_layers(x)
        return x

#loading and preprocessing data
def get_data_loaders(batch_size=64, data_augmentation=True):
    #data augmentation and normalization for training
    #normalize for validation/test
    if data_augmentation:
        train_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomRotation(15),      #rotation
            transforms.RandomVerticalFlip(),    #verticalflip
            transforms.RandomHorizontalFlip(),  #horizontal flip
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    else:
        train_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    train_dataset = ImageFolder(root='/content/drive/MyDrive/inaturalist_12K/train', transform=train_transform)
    test_dataset = ImageFolder(root='/content/drive/MyDrive/inaturalist_12K/val', transform=val_transform)

    #split dataset into train, validation, and test sets
    train_size = int(0.8 * len(train_dataset))
    val_size = len(train_dataset) - train_size
    train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    return train_loader, val_loader, test_loader

#train the model
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device='cuda'):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        #Validation
        model.eval()
        total = 0
        correct = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = 100 * correct / total
        wandb.log({'epoch': epoch, 'val_accuracy': val_accuracy})
        print(f'Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {val_accuracy:.2f}%')


#test the model
def test(model, test_loader, device='cuda'):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    test_accuracy = 100 * correct / total
    wandb.log({'test_accuracy': test_accuracy})
    return test_accuracy

sweep_config = {
        'method': 'bayes', #try for grid,random and bayes
        'metric': {'name': 'val_accuracy', 'goal': 'maximize'},
        'parameters': {
            'num_filters': {'values': [32, 64]},
            'filter_size': {'values': [3, 5]},
            'activation': {'values': ['relu', 'gelu', 'silu', 'mish']},
            'data_augmentation': {'values': [True, False]},
            'batch_norm': {'values': [True, False]},
            'dropout_prob': {'values': [0.0, 0.2, 0.3]},
        }
    }


#the sweep
sweep_id = wandb.sweep(sweep=sweep_config, project='DLassignment2')

#hyperparameters
def main():
    with wandb.init() as run:
        config = wandb.config

        train_loader, val_loader, test_loader = get_data_loaders(data_augmentation=config.data_augmentation)

        model = CNN(10, config.num_filters, config.filter_size, config.dropout_prob, config.activation, use_batchnorm=config.batch_norm)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        train(model, train_loader, val_loader, criterion, optimizer)

        test_accuracy = test(model, test_loader)
        print(f'Test Accuracy: {test_accuracy:.2f}%')

#run the sweep agent
wandb.agent(sweep_id, function=main, count=10)


#references:
#1.https://discuss.pytorch.org/t/correct-way-to-calculate-train-and-valid-loss/178974
#2.https://blog.paperspace.com/writing-cnns-from-scratch-in-pytorch/
#3.https://medium.com/swlh/deep-learning-for-image-classification-creating-cnn-from-scratch-using-pytorch-d9eeb7039c12
#4.https://www.kdnuggets.com/building-a-convolutional-neural-network-with-pytorch
#5.https://machinelearningmastery.com/building-a-convolutional-neural-network-in-pytorch/
#6.https://pytorch.org/docs/stable/index.html

In [ ]:
!pip install wandb
import wandb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import numpy as np
import imageio
import os
import cv2
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
wandb.login(key='e3c892d4f8c9cd9b9043d31938ad090f0a32cec1')

#CNN model
class CNN(nn.Module):
    def __init__(self, num_classes, num_filters=32, filter_size=3, dropout_prob=0.0, activation='relu', use_batchnorm=True):
        super(CNN, self).__init__()
        #activation functions
        activation_fn = nn.ReLU() if activation == 'relu' else nn.GELU() if activation == 'gelu' else \
                        nn.SiLU() if activation == 'silu' else nn.Mish()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, num_filters, kernel_size=filter_size, padding='same'),
            nn.BatchNorm2d(num_filters) if use_batchnorm else nn.Identity(),    #batch normalization
            activation_fn,
            nn.MaxPool2d(2,2),
            nn.Conv2d(num_filters, num_filters*2, kernel_size=filter_size, padding='same'),
            nn.BatchNorm2d(num_filters*2) if use_batchnorm else nn.Identity(),
            activation_fn,
            nn.MaxPool2d(2,2),
            nn.Conv2d(num_filters*2, num_filters*4, kernel_size=filter_size, padding='same'),
            nn.BatchNorm2d(num_filters*4) if use_batchnorm else nn.Identity(),
            activation_fn,
            nn.MaxPool2d(2,2),
            nn.Conv2d(num_filters*4, num_filters*8, kernel_size=filter_size, padding='same'),
            nn.BatchNorm2d(num_filters*8) if use_batchnorm else nn.Identity(),
            activation_fn,
            nn.MaxPool2d(2,2),
            nn.Conv2d(num_filters*8, num_filters*16, kernel_size=filter_size, padding='same'),
            nn.BatchNorm2d(num_filters*16) if use_batchnorm else nn.Identity(),
            activation_fn,
            nn.MaxPool2d(2,2)
        )
         #fully connected layers
        self.fc_layers = nn.Sequential(
            nn.Linear(num_filters*16 * 7 * 7, 512),
            activation_fn,
            nn.Dropout(dropout_prob),  #dropout for regularization
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1) #flatten the output for fully connected layers
        x = self.fc_layers(x)
        return x

#loading and preprocessing data
def get_data_loaders(batch_size=64, data_augmentation=True):
    #data augmentation and normalization for training
    #normalize for test
    if data_augmentation:
        train_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomRotation(15),      #rotation
            transforms.RandomVerticalFlip(),    #verticalflip
            transforms.RandomHorizontalFlip(),  #horizontal flip
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    else:
        train_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    train_dataset = ImageFolder(root='/content/drive/MyDrive/inaturalist_12K/train', transform=train_transform)
    test_dataset = ImageFolder(root='/content/drive/MyDrive/inaturalist_12K/val', transform=test_transform)


    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    return train_loader,test_loader


def evaluate(model, train_loader, test_loader, criterion, optimizer,device='cuda', num_epochs=10,):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        #Validation
        model.eval()
        total = 0
        correct = 0
        predictions = []
        ground_truth = []

        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                predictions.extend(predicted.cpu().numpy())
                ground_truth.extend(labels.cpu().numpy())
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        test_accuracy = 100 * correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Test Accuracy: {test_accuracy:.2f}%')
    return predictions, ground_truth

train_loader,test_loader = get_data_loaders(64,True)
model = CNN(10, 32, 3, 0.0, "silu", False)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
predictions, ground_truth = evaluate(model,train_loader, test_loader, criterion, optimizer)

#Sample images and predictions
sample_indices = np.random.choice(len(predictions), size=30, replace=False)
sample_images = []
for index in sample_indices:
    sample_images.append(test_loader.dataset.imgs[index][0])

#WandB
wandb.init(project="DLassignment2")

for i, index in enumerate(sample_indices):
    img = sample_images[i]
    prediction = predictions[index]
    true_label = ground_truth[index]
    wandb.log({"image": wandb.Image(img, caption=f"Prediction: {prediction}, True Label: {true_label}")})


wandb.finish()


In [ ]:
!pip install wandb

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import wandb


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

wandb.login(key='e3c892d4f8c9cd9b9043d31938ad090f0a32cec1')


def get_data_loaders(batch_size=64):
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    train_dataset = ImageFolder(root='/content/drive/MyDrive/inaturalist_12K/train', transform=train_transform)
    test_dataset = ImageFolder(root='/content/drive/MyDrive/inaturalist_12K/val', transform=val_transform)


    train_size = int(0.8 * len(train_dataset))
    val_size = len(train_dataset) - train_size
    train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    return train_loader, val_loader, test_loader


def fine_tune_pretrained_model(model, train_loader, val_loader, num_epochs=10, learning_rate=0.001):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)


    for epoch in range(num_epochs):
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()


        model.eval()
        total = 0
        correct = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = 100 * correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {val_accuracy:.2f}%')


sweep_config = {
    'method': 'grid',
    'metric': {'name': 'val_accuracy', 'goal': 'maximize'},
    'parameters': {
        'freeze_percentage': {
            'values': [0.85, 0.90, 0.95]  #freeze 85%, 90%, or 95% of layers
        }
    }
}


sweep_id = wandb.sweep(sweep=sweep_config, project='DLassignment2B')


def main():
    with wandb.init() as run:
        config = wandb.config
        model = models.resnet50(pretrained=True)
        freeze_layers = int(len(list(model.parameters())) * config.freeze_percentage)


        for idx, param in enumerate(model.parameters()):
            if idx < freeze_layers:
                param.requires_grad = False


        train_loader, val_loader, _ = get_data_loaders()


        fine_tune_pretrained_model(model, train_loader, val_loader)


wandb.agent(sweep_id, function=main,count=3)
